**Kaggle > Notebooks > New notebook**
* Notebook : 
* "Add Data" > "Competition Data" > Porto > Search : "porto" > Porto Seguro’s Safe Driver Prediction "+Add" 로 데이타 추가


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
#import pandas as pd
#sample_submission = pd.read_csv("../input/porto-seguro-safe-driver-prediction/sample_submission.csv")
#test = pd.read_csv("../input/porto-seguro-safe-driver-prediction/test.csv")
#train = pd.read_csv("../input/porto-seguro-safe-driver-prediction/train.csv")

=> "porto" 데이타를 추가하면, 여기까지 자동으로 생성됨
=> 아래에 다시 loading 예정

https://www.kaggle.com/bertcarremans/data-preparation-exploration 에 따라,
=> 자동차 및 주택 소유자 보험, 자동차 보험 회사의 클레임 예측이 부정확하면 좋은 운전자 보험 비용이 상승하고, 나쁜 운전가 가격이 낮아짐
=> 내년에 운전자가 자동차 보험 청구를 시작할 확률을 예측하는 모델 구축 
* Visual inspectio of your data
* Defining the metadata
* Descriptive statistics
* Handling imbalanced classes
* Data qualit checks
* Exploratory data visualization
* Feature Engineering
* Feature Selection
* Feature Scaling

**Loading packages**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 100)

**Loading Data**

In [ ]:
train = pd.read_csv('../input/porto-seguro-safe-driver-prediction/train.csv')
test = pd.read_csv('../input/porto-seguro-safe-driver-prediction/test.csv')

**Data as first sight**
* -1(value) : missing ( -1? -> NaN )
* features have similar tag : ps / reg / bin / car / cat
* features name : anonymouse
* postfix "bin" features : binary features
* "cat" features : categorical 
* target column : whether or not a claim 

In [ ]:
train.head()

In [ ]:
# check tail
train.tail()

Have to check
* binary variables
* category variables : integers
* -1(variable) : missing values
* target values / ID variable

**The number of rows and columns**

In [ ]:
train.shape

* 595,212 rows / 59 columns

**Drop duplicate rows in training data**

In [ ]:
train.drop_duplicates()
train.shape

* => No dup rows

**Check number of rows, cols of test set**

In [ ]:
test.shape

* => train : 59 col
* => test : 58 col
* => test에 column이 하나 없다. ( target이 없음. 이런 데이타가 있나...)

In [ ]:
test.head()
# 정말 target column이 없다.
# 주어진 데이타로 complain 하는 건 인지 아닌 건 인지 알고 싶으니.

**look into how many variables of each type have**

In [ ]:
train.info()

* => 14 _cat variables
* => _bin : binary
* ok no null values

**Handling imbalanced class**
* the count of target value?

In [ ]:
print(train.target.unique())

=> target은 0,1로 이뤄져있다. 다행임

In [ ]:
print(sum(train['target'] == 1) / 595212 * 100)

* => train count 595,212 중에 target=1 이 3.6% 다. 
* => targe=0 보다 훨씬 적다. strongly imbalanced.
* => 클레임 예측이 부정확하면(낮다고 보여져서) 좋은 운전자 보험 비용이 상승하고, 나쁜 운전가 가격이 낮아짐
* => undersample or oversample 과정이 필요 ???? 어떻게 하자는걸까? https://www.kaggle.com/bertcarremans/data-preparation-exploration 는 처리하는 내용 있음
* => oversampling records with target=1
* => undersampling records with target=0

**Exploratory Data Visualization**
* => Categorical variables

* => test에는 target이 없다. np.nan 으로 채우자

In [ ]:
test['target']

In [ ]:
test['target'] = np.nan

In [ ]:
test['target']

* => 분석 편의를 위해 변수 유형별로 구분
* => https://www.kaggle.com/bertcarremans/data-preparation-exploration 에서는 데이타 타입을 쉽게 파악하기위해 Metadata 를 만들었지만, 지금 이해가 안가서 skip

**bar_plot 함수를 몇개 만들어서...**

In [ ]:
# column과 data, hue를 받아서, hue값에 따른 seaborn countplot
def bar_plot(col, data, hue=None):
    f, ax = plt.subplots(figsize=(10,5))
    sns.countplot(x=col, hue=hue, data=data, alpha=0.5)
    plt.show()

# histogram, data[col] 사용해서, 해당 컬럼 histogram 
def dist_plot(col, data):
    f, ax = plt.subplots(figsize=(10,5))
    sns.distplot(data[col].dropna(), kde=False, bins=10)
    plt.show()

# target 기준으로 column 값과 target 관계 살펴봄
def bar_plot_ci(col, data):
    f, ax = plt.subplots(figsize=(10,5))
    sns.barplot(x = col, y='target', data=data)
    plt.show()

* 분석 편의로 변수 유형별로 구분 ( train.info() 로 변수 type을 보고, 아래와 같이 나눔 )

In [ ]:
#이진 변수 list
binary = ['ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin',
          'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_calc_15_bin',
          'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin', 'ps_calc_20_bin']

# 범주형 변수
category = ['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat',
            'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat',
            'ps_car_10_cat', 'ps_car_11_cat']

# 정수형 변수
integer = ['ps_ind_01', 'ps_ind_03', 'ps_ind_14', 'ps_ind_15', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
           'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11', 'ps_calc_12', 'ps_calc_13',
           'ps_calc_14', 'ps_car_11']

# 소수형 변수
floats = ['ps_reg_01', 'ps_reg_02', 'ps_reg_03', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_car_12', 'ps_car_13',
          'ps_car_14', 'ps_car_15']

** binary variable**

In [ ]:
for col in binary:
    bar_plot(col, train)

* => 0 value가 많네요

**Category variable**

In [ ]:
for col in category:
    bar_plot(col, train)

* => column 별 category가 어떻게 분포하는지 눈으로 확인 가능

* **integer variable**

In [ ]:
for col in integer:
    bar_plot(col, train)

* **float variable**

In [ ]:
for col in floats:
    bar_plot(col, train)

* corr with heatmap
* => feature가 많아서 corr heatmap 연산 시간 오래 걸림.

In [ ]:
corr_data = train.corr()
f, ax = plt.subplots(figsize=(15,7))
sns.heatmap(corr_data, cmap='summer')

* => Feature가 많아서.... 

**target과 상관관계 살펴보기**
* => binary와 target ....

In [ ]:
for col in binary:
    bar_plot_ci(col, train)

![](https://statkclee.github.io/model/fig/xgboost-python.png)